# Learn ASE's calculator 
- Docs: https://wiki.fysik.dtu.dk/ase/ase/calculators/calculators.html

# Mixed Calculators
https://ase-lib.org/ase/calculators/mixing.html#mixing-calculators

We can combine multiple calculators in simulations, using `ase.calculators.mixing` module.

Example: Use `DFT-D3` for vdW correction with a `DFT` calculator.

```python
from ase.calculators.mixing import SumCalculator
from ase.calculators.dftd3 import DFTD3
from ase.calculators.gpaw import GPAW

calc1 = GPAW(...)
calc2 = DFTD3(...)
calc = SumCalculator([calc1, calc2])
atoms.calc = calc
```

## Results from `SumCalculator` 
How to get separate results from each calculator in `SumCalculator`?

The workaround:
- Make a copy of `Atoms` object.
- Then, use `SinglePointCalculator` to store which calculator's results you want.
- Access the results from each calculators using `atoms.calc.mixer.calcs`, or simply `calc1.results`, `calc2.results`.

In [8]:
from ase.build import molecule
from dftd3.ase import DFTD3
from ase.calculators.emt import EMT
from ase.calculators.mixing import SumCalculator

atoms = molecule("CH3CH2OCH3")
atoms.center(vacuum=5)
calc1 = EMT()
calc2 = DFTD3(method="PBE", damping="d3zero")
atoms.calc = SumCalculator([calc1, calc2])

energy = atoms.get_potential_energy()
forces = atoms.get_forces()
# stress = atoms.get_stress()
atoms.calc.results  # total results from both calculators

{'energy_contributions': [np.float64(4.248476802164584),
  np.float64(-0.0804729011506879)],
 'energy': np.float64(4.168003901013896),
 'forces_contributions': [array([[ 1.12381667e+00,  7.47837920e-01,  1.11022302e-16],
         [-3.59453540e+00, -2.83390268e+00,  1.77635684e-15],
         [-3.67604734e+00,  1.87168675e+00,  2.66453526e-15],
         [ 3.62917712e+00, -1.11569676e+00,  0.00000000e+00],
         [ 1.34557997e+00,  9.92459565e-01,  2.19740343e+00],
         [ 1.34557997e+00,  9.92459565e-01, -2.19740343e+00],
         [-1.36682553e-01, -2.70621940e+00,  2.08171052e-17],
         [ 1.26626450e+00,  6.82819487e-01, -1.95088959e+00],
         [ 1.26626450e+00,  6.82819487e-01,  1.95088959e+00],
         [-1.20453688e+00, -8.37030229e-01, -2.00634032e+00],
         [-1.20453688e+00, -8.37030229e-01,  2.00634032e+00],
         [-1.60343663e-01,  2.35979652e+00,  0.00000000e+00]]),
  array([[-2.49943502e-02, -5.19719115e-03, -8.98210203e-18],
         [ 2.09143314e-03,  1.702

In [48]:
atoms.calc.mixer.calcs

In [59]:
calc1.results

{'energy': np.float64(4.248476802164584),
 'free_energy': np.float64(4.248476802164584),
 'energies': array([ 0.36398294, -0.46300406, -0.63315988, -0.49539208,  0.69845806,
         0.69845806,  0.70278893,  0.666097  ,  0.666097  ,  0.6865866 ,
         0.6865866 ,  0.67097763]),
 'forces': array([[ 1.12381667e+00,  7.47837920e-01,  1.11022302e-16],
        [-3.59453540e+00, -2.83390268e+00,  1.77635684e-15],
        [-3.67604734e+00,  1.87168675e+00,  2.66453526e-15],
        [ 3.62917712e+00, -1.11569676e+00,  0.00000000e+00],
        [ 1.34557997e+00,  9.92459565e-01,  2.19740343e+00],
        [ 1.34557997e+00,  9.92459565e-01, -2.19740343e+00],
        [-1.36682553e-01, -2.70621940e+00,  2.08171052e-17],
        [ 1.26626450e+00,  6.82819487e-01, -1.95088959e+00],
        [ 1.26626450e+00,  6.82819487e-01,  1.95088959e+00],
        [-1.20453688e+00, -8.37030229e-01, -2.00634032e+00],
        [-1.20453688e+00, -8.37030229e-01,  2.00634032e+00],
        [-1.60343663e-01,  2.3597965

In [60]:
calc2.results

{'energy': np.float64(-0.0804729011506879),
 'free_energy': np.float64(-0.0804729011506879),
 'forces': array([[-2.49943502e-02, -5.19719115e-03, -8.98550486e-18],
        [ 2.09143314e-03,  1.70202644e-03, -3.45608120e-18],
        [-1.30972918e-03, -4.92022551e-03, -1.28490746e-18],
        [-4.73357360e-03,  1.45612542e-03, -5.13372806e-18],
        [ 2.91645062e-03,  9.50171045e-03,  5.86830294e-03],
        [ 2.91645062e-03,  9.50171045e-03, -5.86830294e-03],
        [ 1.12084405e-03, -2.64150627e-03, -2.89044629e-18],
        [ 8.59261134e-03, -1.13676445e-03, -5.73199609e-03],
        [ 8.59261134e-03, -1.13676445e-03,  5.73199609e-03],
        [ 2.11374948e-03, -3.96919463e-03, -1.16851843e-03],
        [ 2.11374948e-03, -3.96919463e-03,  1.16851843e-03],
        [ 5.79752865e-04,  8.09268328e-04, -1.48242030e-18]])}

*Note that*: `calc2.results` give exactly the same results as `DFTD3` calculator when used alone. (see below)

Now write only the results from `calc1`

In [10]:
from ase.calculators.singlepoint import SinglePointCalculator

atoms1 = atoms.copy()
# atoms1.calc = calc1
atoms1.calc = SinglePointCalculator(atoms1)  # use this safer than above line
atoms1.calc.results = calc1.results
# atoms1.write("atoms1.extxyz", format="extxyz")  # only EMT results

In [11]:
atoms1.calc.results

{'energy': np.float64(4.248476802164584),
 'free_energy': np.float64(4.248476802164584),
 'energies': array([ 0.36398294, -0.46300406, -0.63315988, -0.49539208,  0.69845806,
         0.69845806,  0.70278893,  0.666097  ,  0.666097  ,  0.6865866 ,
         0.6865866 ,  0.67097763]),
 'forces': array([[ 1.12381667e+00,  7.47837920e-01,  1.11022302e-16],
        [-3.59453540e+00, -2.83390268e+00,  1.77635684e-15],
        [-3.67604734e+00,  1.87168675e+00,  2.66453526e-15],
        [ 3.62917712e+00, -1.11569676e+00,  0.00000000e+00],
        [ 1.34557997e+00,  9.92459565e-01,  2.19740343e+00],
        [ 1.34557997e+00,  9.92459565e-01, -2.19740343e+00],
        [-1.36682553e-01, -2.70621940e+00,  2.08171052e-17],
        [ 1.26626450e+00,  6.82819487e-01, -1.95088959e+00],
        [ 1.26626450e+00,  6.82819487e-01,  1.95088959e+00],
        [-1.20453688e+00, -8.37030229e-01, -2.00634032e+00],
        [-1.20453688e+00, -8.37030229e-01,  2.00634032e+00],
        [-1.60343663e-01,  2.3597965

# Calculators for `DFT-D3` vdW correction
There are many packages for `DFT-D3` vdW correction calculator in ASE. Some of them support GPU acceleration.

Why need DFT-D3? -> Read the blog: https://tech.preferred.jp/en/blog/oss-pytorch-dftd3/

Also read the note: [../0_GPAW/gpaw1_vdW_correction.ipynb](../0_GPAW/gpaw1_vdW_correction.ipynb)

Let see some of them. There are 2 best candidates:
- `simple-dftd3` (CPU only), and `torch-dftd3` (CPU and GPU)
- The resutls are almost the same (slightly different maybe due different default parameters)
- `torch-dftd3` is much faster, specially for large system.

In [ ]:
from ase.build import molecule
from ase.visualize import view

atoms = molecule("CH3CH2OCH3")
atoms.center(vacuum=5)
view(atoms)

<Popen: returncode: None args: ['/home/tha/app/miniforge/envs/py13/bin/pytho...>

## Built-in DFTD3 in ASE
NOTE: Implemation in ASE require compile the DFTD3 code separately. It is not included in the ASE package.

The [DFTD3](https://www.chemie.uni-bonn.de/grimme/de/software/dft-d3/) calculator in ASE

## `simple-dftd3` package (CPU only)
- pip: https://pypi.org/project/dftd3/
- docs: https://dftd3.readthedocs.io/en/latest/index.html
- ASE integrate: https://github.com/dftd3/simple-dftd3/blob/main/python/dftd3/ase.py

damping options: `d3zero`, `d3zerom`, `d3bj`, `d3bjm`, `d3op`. See https://github.com/dftd3/simple-dftd3/blob/main/python/dftd3/ase.py#L111

Install
```bash
pip install dftd3
conda install -c conda-forge dftd3-python
``` 

In [52]:
from dftd3.ase import DFTD3

atoms = molecule("CH3CH2OCH3")
atoms.center(vacuum=5)
atoms.calc = DFTD3(method="PBE", damping="d3zero")

energy = atoms.get_potential_energy()
forces = atoms.get_forces()
print("energy:\n", energy)
print("forces:\n", forces)

energy:
 -0.0804729011506879
forces:
 [[-2.49943502e-02 -5.19719115e-03 -8.98482430e-18]
 [ 2.09143314e-03  1.70202644e-03 -3.38172943e-18]
 [-1.30972918e-03 -4.92022551e-03 -1.30668555e-18]
 [-4.73357360e-03  1.45612542e-03 -5.08295947e-18]
 [ 2.91645062e-03  9.50171045e-03  5.86830294e-03]
 [ 2.91645062e-03  9.50171045e-03 -5.86830294e-03]
 [ 1.12084405e-03 -2.64150627e-03 -2.97619753e-18]
 [ 8.59261134e-03 -1.13676445e-03 -5.73199609e-03]
 [ 8.59261134e-03 -1.13676445e-03  5.73199609e-03]
 [ 2.11374948e-03 -3.96919463e-03 -1.16851843e-03]
 [ 2.11374948e-03 -3.96919463e-03  1.16851843e-03]
 [ 5.79752865e-04  8.09268328e-04 -1.48173973e-18]]


In [ ]:
%%timeit
energy = atoms.get_potential_energy()
forces = atoms.get_forces()

96.2 μs ± 470 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## `torch-dftd` package (GPU/CPU)
- GitHub: https://github.com/pfnet-research/torch-dftd
- Docs: https://tech.preferred.jp/en/blog/oss-pytorch-dftd3/

This package may have more features than `simple-dftd3`.

damping method. Choices: "zero", "bj", "zerom", "bjm". See all parameters of `torch-dftd3`: https://github.com/pfnet-research/torch-dftd/blob/master/torch_dftd/torch_dftd3_calculator.py#L17

```bash
pip install torch-dftd
```

In [38]:
from torch_dftd.torch_dftd3_calculator import TorchDFTD3Calculator

atoms = molecule("CH3CH2OCH3")
atoms.center(vacuum=50)
# device="cuda:0" for fast GPU computation.
atoms.calc = TorchDFTD3Calculator(device="cpu", damping="zero")

energy = atoms.get_potential_energy()
forces = atoms.get_forces()
print("energy:\n", energy)
print("forces:\n", forces)

energy:
 -0.08060236183241912
forces:
 [[-2.50326227e-02 -5.34063205e-03  9.26720745e-09]
 [ 2.10378878e-03  1.75449520e-03 -1.04496705e-08]
 [-1.29321474e-03 -4.95797023e-03  1.49870272e-09]
 [-4.77362843e-03  1.46504515e-03 -8.25661495e-09]
 [ 2.97987252e-03  9.58887115e-03  5.96211106e-03]
 [ 2.97986763e-03  9.58886929e-03 -5.96210686e-03]
 [ 1.01956818e-03 -2.59048585e-03  1.78744367e-10]
 [ 8.55891872e-03 -1.09402917e-03 -5.76877268e-03]
 [ 8.55891779e-03 -1.09402905e-03  5.76877315e-03]
 [ 2.11785152e-03 -4.03105887e-03 -1.20648509e-03]
 [ 2.11784756e-03 -4.03107610e-03  1.20649592e-03]
 [ 6.62833510e-04  7.42002914e-04 -8.19474177e-09]]


In [ ]:
%%timeit
energy = atoms.get_potential_energy()
forces = atoms.get_forces()

95.7 μs ± 387 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Whether GPU is faster?

In [ ]:
%%timeit
atoms.calc = TorchDFTD3Calculator(device="cuda:0", damping="zero")

energy = atoms.get_potential_energy()
forces = atoms.get_forces()

13.2 ms ± 323 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [17]:
help(TorchDFTD3Calculator)

Help on class TorchDFTD3Calculator in module torch_dftd.torch_dftd3_calculator:

class TorchDFTD3Calculator(ase.calculators.calculator.Calculator)
 |  TorchDFTD3Calculator(
 |      dft: Optional[ase.calculators.calculator.Calculator] = None,
 |      atoms: ase.atoms.Atoms = None,
 |      damping: str = 'zero',
 |      xc: str = 'pbe',
 |      old: bool = False,
 |      device: str = 'cpu',
 |      cutoff: float = 50.27183500356491,
 |      cnthr: float = 21.167088422553647,
 |      abc: bool = False,
 |      dtype: torch.dtype = torch.float32,
 |      bidirectional: bool = True,
 |      cutoff_smoothing: str = 'none',
 |      **kwargs
 |  )
 |
 |  ase compatible DFTD3 calculator using pytorch
 |
 |  Args:
 |      dft (Calculator or None): base dft calculator can be set here
 |      atoms (Atoms):
 |      damping (str): damping method. "zero", "bj", "zerom", "bjm"
 |      xc (str): exchange correlation functional
 |      old (bool): Use DFTD2 method when `True`, DFTD3 method is used whe

## `sevennet` package (GPU only)
- GitHub: https://github.com/MDIL-SNU/SevenNet/blob/main/sevenn/calculator.py#L330

Limite features.